Are there sites inaccessible from one technology / technique vs another?
  - I bet there are sites where non-nist WGS all miss but every TR caller captures
  - I bet there are sites long reads capture but none of the short reads capture
  
Take the refine.regions.txt from all tools and combine like how I'm doing for Q4.
Perform queries based on path.txt metadata's sequencing,  Caller Type,  Sequencing, Locations.

Where do each subset all agree / disagree with one another.

wgs vs catalog (pretty much same as TR vs non-TR). 
Short-reads vs long reads


In [2]:
import os
import joblib
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import json

In [3]:
def region_loader(path, name):
    full_path = os.path.join(path, 'refine.regions.txt')
    benchmark_regions = pd.read_csv(full_path, 
                                #names=["chrom", "start", "end", "tier", "repl", "vflag", "entropy", "ad1", "ad2"],
                                sep='\t').set_index(["chrom", "start", "end"])
    benchmark_regions.columns = [f'{c}_{name}' for c in benchmark_regions.columns]
    return benchmark_regions
paths = pd.read_csv("paths.txt", sep='\t')
paths = paths[paths['program'].isin(['biograph', 
                                     'deepvariant', 
                                     'gangstr', 
                                     'hipstr', 
                                     'medakaTR', 
                                     'sniffles', 
                                     'trgt', 
                                     'GIABv4.2.1'])]
parts = []
for _, i in paths.iterrows():
    parts.append(region_loader(i['path'], i['program']))
data = pd.concat(parts, axis=1)

In [4]:
# When doing the agreement, we want for all tier1 sites as well as for all >=5bp
# Lets repeat all of that with Tier1

benchmark_regions = pd.read_csv("/Users/english/code/adotto/benchmark/GIABTR_benchmark.6.26/GIABTR.HG002.benchmark.regions.bed.gz", 
                                names=["chrom", "start", "end", "tier", "repl", "vflag", "entropy", "ad1", "ad2"],
                                sep='\t').set_index(["chrom", "start", "end"])

benchmark_regions['max_allele_delta'] = benchmark_regions[['ad1', 'ad2']].max(axis=1)
s = benchmark_regions.reset_index()
x = (s['end'] - s['start']).to_frame().set_index(benchmark_regions.index)
benchmark_regions['span'] = x

# Want to load the catalog and also do this grouping for the mean purity of the repeats
catalog = (pd.read_csv("/Users/english/code/adotto/regions/adotto_TRregions_v1.1.bed", sep='\t')
           .rename(columns={'chr':'chrom'})
           .set_index(['chrom', 'start', 'end']))


def max_motif(x):
    d = json.loads(x)
    return max([_['period'] for _ in d])
catalog['max_motif'] = catalog['annos'].apply(max_motif)


benchmark_regions = benchmark_regions.join(catalog[['mu_purity', 'max_motif']])

t1_index = benchmark_regions[benchmark_regions['tier'] == 'Tier1'].index

def is_gt5(x):
    return (x & 0x1) != 0
gt5_index = benchmark_regions[(benchmark_regions['tier'] == 'Tier1') & benchmark_regions['vflag'].apply(is_gt5)].index


In [5]:
to_analyze = ~data['state_hipstr'].isna()

In [6]:
# How many are we subsetting to?
len(benchmark_regions[benchmark_regions.index.isin(to_analyze[to_analyze].index)
                      & benchmark_regions.index.isin(gt5_index)])

60325

In [7]:
to_analyze.value_counts()

True     1058986
False     647867
Name: state_hipstr, dtype: int64

In [8]:
data[[_ for _ in data.columns if _.startswith('state')]].isna().sum(axis=0)

state_biograph            0
state_deepvariant         0
state_gangstr             0
state_medakaTR            0
state_sniffles            0
state_GIABv4.2.1          0
state_trgt                0
state_hipstr         647867
dtype: int64

In [9]:
def within_set_agree(m_data, m_set):
    # Do the state_name agree / disagree within a set. Returns a True/False
    return m_data['state_' + m_set[0]] == m_data['state_' + m_set[1]]

sr_callers_agree = within_set_agree(data, ['gangstr', 'hipstr'])
lr_callers_agree = within_set_agree(data, ['trgt', 'medakaTR'])

In [10]:
sr_callers_agree[sr_callers_agree.index.isin(t1_index)
                & sr_callers_agree.index.isin(gt5_index)
                & to_analyze].value_counts()

True     41647
False    18678
dtype: int64

In [11]:
sr_callers_agree[sr_callers_agree.index.isin(t1_index)
                & sr_callers_agree.index.isin(gt5_index)
                & to_analyze].mean()

0.6903771239121426

In [12]:
lr_callers_agree[lr_callers_agree.index.isin(t1_index)
                & lr_callers_agree.index.isin(gt5_index)
                & to_analyze].value_counts()

True     58772
False     1553
dtype: int64

In [13]:
lr_callers_agree[lr_callers_agree.index.isin(t1_index)
                & lr_callers_agree.index.isin(gt5_index)
                & to_analyze].mean()

0.9742561127227518

In [14]:
v = data[sr_callers_agree
        & lr_callers_agree
        & data.index.isin(t1_index)
        & data.index.isin(gt5_index)
        & to_analyze][['state_hipstr', 'state_trgt']].value_counts().unstack().fillna(0).astype(int)
v

state_trgt,FN,"FN,FP",FP,TN,TP
state_hipstr,,,,,
FN,56,11,22,25,12571
"FN,FP",2,6,0,1,105
FP,0,0,34,0,32
TN,0,0,0,1,0
TP,4,1,0,3,27806


In [15]:
v.loc[:"FP"][["TN", "TP"]]

state_trgt,TN,TP
state_hipstr,,
FN,25,12571
"FN,FP",1,105
FP,0,32


In [16]:
v.loc[["TN", "TP"], :"FP"]

state_trgt,FN,"FN,FP",FP
state_hipstr,,,
TN,0,0,0
TP,4,1,0


In [17]:
v.loc[:"FP", :"FP"]

state_trgt,FN,"FN,FP",FP
state_hipstr,,,
FN,56,11,22
"FN,FP",2,6,0
FP,0,0,34


In [18]:
tmp = pd.DataFrame([["both", v.loc[("TP", "TP")] + v.loc[("TN", "TN")]],
                    ["only_lr", v.loc[:"FP"][["TN", "TP"]].sum().sum()],
                    ["only_sr", v.loc[["TN", "TP"], :"FP"].sum().sum()]],
                    #['neither', v.loc[:"FP", :"FP"].sum().sum()]],
                   columns=['which', 'cnt'])
tmp['pct'] = tmp['cnt'] /  tmp['cnt'].sum()
print(tmp['cnt'].sum())
tmp

40546


,which,cnt,pct
0,both,27807,0.685814
1,only_lr,12734,0.314063
2,only_sr,5,0.000123


In [19]:
# What are lr getting that sr aren't?

subset = (to_analyze 
          & sr_callers_agree 
          & lr_callers_agree 
          & data.index.isin(t1_index) 
          & data.index.isin(gt5_index))

both_mask = data[subset]['state_hipstr'].str.startswith('T') & data[subset]['state_trgt'].str.startswith('T')
lr_mask = ~data[subset]['state_hipstr'].str.startswith('T') & data[subset]['state_trgt'].str.startswith('T')

lro = benchmark_regions.loc[lr_mask[lr_mask].index]
both = benchmark_regions.loc[both_mask[both_mask].index]


In [20]:
a = lro[['max_allele_delta', 'entropy', 'span', 'mu_purity', 'max_motif']].describe().T
a = a.set_index(pd.MultiIndex.from_tuples([(_, 'lr_only') for _ in a.index]))
b = both[['max_allele_delta', 'entropy', 'span', 'mu_purity', 'max_motif']].describe().T
b = b.set_index(pd.MultiIndex.from_tuples([(_, 'both') for _ in b.index]))
view = pd.concat([a, b]).sort_index()
view

count        mean         std        min  \
entropy          both     27810.0    0.889635    0.054261   0.480618   
                 lr_only  12734.0    0.882020    0.094677   0.411906   
max_allele_delta both     27810.0   11.505574   14.770920   5.000000   
                 lr_only  12734.0  124.202843  471.742744   5.000000   
max_motif        both     27810.0    6.676232   26.134335   2.000000   
                 lr_only  12734.0   43.054421   74.758963   2.000000   
mu_purity        both     27810.0   95.464545    2.448179  79.000000   
                 lr_only  12734.0   91.728522    2.577267  76.000000   
span             both     27810.0  125.231607  150.999614  27.000000   
                 lr_only  12734.0  593.328019  689.706315  20.000000   

                                 25%         50%         75%           max  
entropy          both       0.856296    0.893779    0.927436      0.999828  
                 lr_only    0.826462    0.899770    0.965586      0.999930  
max_allele_delta both       6.000000   10.000000   14.000000   1588.000000  
                 lr_only   10.000000   23.000000   68.000000  17516.000000  
max_motif        both       2.000000    3.000000    5.000000    481.000000  
                 lr_only    6.000000   16.000000   38.000000    495.000000  
mu_purity        both      94.000000   96.000000   97.000000    100.000000  
                 lr_only   90.000000   92.000000   94.000000     99.000000  
span             both      83.000000   93.000000  107.000000   3289.000000  
                 lr_only  219.000000  437.000000  772.000000  23939.000000

In [21]:
# Ranksum
from scipy.stats import ranksums

check = [('greater', 'max_allele_delta'),
         ('greater', 'span'),
         ('greater', 'max_motif'),
         ('less', 'mu_purity'),
         ('less', 'entropy')]
rows = []
for alt, c in check:
    result = ranksums(lro[c], both[c], alternative=alt)
    lstat = lro[c].describe()
    bstat = both[c].describe()
    rows.append([c, 
                 bstat['mean'], bstat['std'], bstat['50%'],
                 lstat['mean'], lstat['std'], lstat['50%'],
                 alt, result.pvalue, result.statistic])
result = pd.DataFrame(rows, columns=['annotation', 'both mean', 'both std', 'both median',
                                     'long-read only mean', 'long-read only std', 'long-read only median',
                                     'alternate', 'p-value', 'statistic'])
result.set_index('annotation')

,both mean,both std,both median,long-read only mean,long-read only std,long-read only median,alternate,p-value,statistic
annotation,,,,,,,,,
max_allele_delta,11.505574,14.770920,10.000000,124.202843,471.742744,23.00000,greater,0.0,83.864245
span,125.231607,150.999614,93.000000,593.328019,689.706315,437.00000,greater,0.0,132.811310
max_motif,6.676232,26.134335,3.000000,43.054421,74.758963,16.00000,greater,0.0,121.863419
mu_purity,95.464545,2.448179,96.000000,91.728522,2.577267,92.00000,less,0.0,-114.142397
entropy,0.889635,0.054261,0.893779,0.882020,0.094677,0.89977,less,1.0,6.645997


# TR vs WGS caller

In [22]:
any_hip = data['state_hipstr'].str.startswith('T')
any_gang = data['state_gangstr'].str.startswith('T')
any_trgt = data['state_trgt'].str.startswith('T')
any_medaka = data['state_medakaTR'].str.startswith('T')
any_bg = data['state_biograph'].str.startswith('T')
any_snif = data['state_sniffles'].str.startswith('T')
any_dv = data['state_deepvariant'].str.startswith('T')

wgs_tr = any_dv | any_bg | any_snif
the_tr = any_hip | any_gang | any_trgt | any_medaka

In [23]:
len(gt5_index) # How many regions are we dealing with?

101704

 Of the benchmark’s 101,704 Tier1 HG002 ≥5bp regions, the TR callers had at least one TP state in 101,1080 (99.4%) regions and 90,996 (89.4%) regions had a TP state from any WGS caller. Intersecting the two sets we found 90,812 (89.2%) are TP in the sets’ union, 10,296 (10.1%) are found to be TP exclusively by the TR callers, and only 184 are exclusive to the WGS callers. Interestingly, both the long-read and short-read TR callers contribute to the set of regions found exclusively by TR callers at 99.8% and 3.4% of regions, respectively. 

In [24]:
# TR had at least one TP:
the_tr[the_tr.index.isin(gt5_index)].value_counts()

True     101124
False       580
dtype: int64

In [26]:
101124 / 101704

0.9942971761189334

In [28]:
# WGS had at least one TP:
wgs_tr[wgs_tr.index.isin(gt5_index)].value_counts()

True     90996
False    10708
dtype: int64

In [29]:
90996 / 101704

0.8947140722095492

In [30]:
(the_tr[the_tr.index.isin(gt5_index)] & wgs_tr[wgs_tr.index.isin(gt5_index)]).value_counts()

True     90818
False    10886
dtype: int64

In [31]:
90818 / 101704

0.8929638952253599

In [32]:
(the_tr[the_tr.index.isin(gt5_index)] & ~wgs_tr[wgs_tr.index.isin(gt5_index)]).value_counts()

False    91398
True     10306
dtype: int64

In [33]:
(~the_tr[the_tr.index.isin(gt5_index)] & wgs_tr[wgs_tr.index.isin(gt5_index)]).value_counts()

False    101526
True        178
dtype: int64

In [34]:
tr_exclusive = the_tr & ~wgs_tr

In [35]:
tr_exclusive[tr_exclusive.index.isin(gt5_index)].value_counts() # 10296

False    91398
True     10306
dtype: int64

In [36]:
sr_contribute = (tr_exclusive & (any_hip | any_gang))
sr_contribute[sr_contribute.index.isin(gt5_index)].value_counts()

False    101305
True        399
dtype: int64

In [37]:
(tr_exclusive & (any_trgt[any_trgt.index.isin(gt5_index)] | any_medaka[any_medaka.index.isin(gt5_index)])).value_counts()

False    1696571
True       10282
dtype: int64

In [38]:
print(399 / 10306)
print(10282 / 10306)

0.03871531146904716
0.9976712594605085
